# Web Scraping de Tabelas (Salvando em CSV)

### Como usar o fluxo:

1. O script irá listar as prévias de todas as tabelas com seus respectivos índices (Ex: [0], [1], [2]...).

2. Quando solicitado, você pode digitar:

- Um único número: 4

- Múltiplos números: 1, 3, 7

- Números duplicados (serão ignorados): 0, 2, 0

3. O script salvará cada tabela escolhida com um nome único, seguindo o padrão: tabela_copa_do_mundo_X.csv (onde X é o índice da tabela).

In [26]:
import pandas as pd
import requests
from io import StringIO
import sys

In [27]:
# Variáveis Globais (Constantes)
URL_PAGINA = "https://pt.wikipedia.org/wiki/Copa_do_Mundo_FIFA"
HEADERS = {"User-Agent": "Mozilla/5.0"}
ARQUIVO_BASE_SAIDA = "tabela_copa_do_mundo" # Base para o nome do arquivo

In [28]:
def main():
    try:
        print("Iniciando requisição à página...")
        response = requests.get(URL_PAGINA, headers=HEADERS, timeout=15)
        response.raise_for_status()
        html_content = response.text

        # Leitura das Tabelas
        html_file_like = StringIO(html_content) 
        tabelas = pd.read_html(html_file_like, header=0, encoding='utf-8') 
        
        num_tabelas = len(tabelas)
        print(f"Total de {num_tabelas} tabelas (DataFrames) encontradas na página.\n")

        if num_tabelas == 0:
            print("Nenhuma tabela foi encontrada na página. Encerrando.")
            return 

        # 4. Loop de Inspeção
        print("--- PRÉVIAS DAS TABELAS ENCONTRADAS ---")
        for i, df in enumerate(tabelas):
            print("="*60)
            print(f"[{i}] Tabela (DataFrame) {i}:")
            print(f"Dimensões: {df.shape} (Linhas x Colunas)")
            
            print("\n--- PRÉVIA DO CABEÇALHO ---")
            print(df.head(3).to_string())
            print("---------------------------\n")

        # 5. Escolha Interativa de Múltiplas Tabelas
        indices_escolhidos = []
        while not indices_escolhidos:
            escolha_raw = input(f"Digite os NÚMEROS das tabelas que deseja salvar, separados por vírgula (ex: 0, 2, 5), ou 'N' para sair: ").strip()
            
            if escolha_raw.upper() == 'N':
                print("Operação cancelada pelo usuário. Encerrando.")
                return

            try:
                # Processa a string de entrada para obter uma lista de índices
                partes = [p.strip() for p in escolha_raw.split(',')]
                
                indices = []
                for p in partes:
                    if p.isdigit():
                        indice = int(p)
                        if 0 <= indice < num_tabelas:
                            indices.append(indice)
                        else:
                            print(f"Aviso: O índice '{p}' está fora do alcance (0 a {num_tabelas - 1}) e será ignorado.")
                    else:
                         print(f"Aviso: '{p}' não é um número válido e será ignorado.")

                if indices:
                    indices_escolhidos = list(set(indices)) # Remove duplicatas
                else:
                    print("Nenhum índice válido foi selecionado. Tente novamente.")

            except Exception:
                print("Erro ao processar a entrada. Certifique-se de usar números separados por vírgula.\n")

        # 6. Salvar as Tabelas Escolhidas em CSV
        print("\n--- SALVANDO TABELAS SELECIONADAS ---")
        for indice in indices_escolhidos:
            df_final = tabelas[indice]
            
            # Cria um nome de arquivo único para cada tabela
            nome_arquivo_saida = f"{ARQUIVO_BASE_SAIDA}_{indice}.csv"
            
            df_final.to_csv(nome_arquivo_saida, index=False, encoding='utf-8')
            
            print(f"SUCESSO: Tabela [{indice}] salva em '{nome_arquivo_saida}' (Linhas: {df_final.shape[0]}).")
            
        print("\nProcesso de salvamento concluído para todas as tabelas selecionadas.")

    except requests.exceptions.RequestException as e:
        print(f"ERRO de requisição: Não foi possível acessar a URL. Detalhe: {e}")
    except ValueError as e:
        print(f"ERRO de parsing: O Pandas não conseguiu encontrar ou analisar as tabelas. Detalhe: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

if __name__ == "__main__":
    main()

Iniciando requisição à página...
Total de 29 tabelas (DataFrames) encontradas na página.

--- PRÉVIAS DAS TABELAS ENCONTRADAS ---
[0] Tabela (DataFrame) 0:
Dimensões: (19, 2) (Linhas x Colunas)

--- PRÉVIA DO CABEÇALHO ---
  Campeonato Mundial de Futebol            Campeonato Mundial de Futebol.1
0            Copa do Mundo FIFA                         Copa do Mundo FIFA
1                  Dados gerais                               Dados gerais
2                   Organização  Federação Internacional de Futebol (FIFA)
---------------------------

[1] Tabela (DataFrame) 1:
Dimensões: (3, 2) (Linhas x Colunas)

--- PRÉVIA DO CABEÇALHO ---
    Dados históricos  Dados históricos.1
0  Primeiro vencedor      Uruguai (1930)
1    Último vencedor    Argentina (2022)
2     Maior vencedor  Brasil (5 títulos)
---------------------------

[2] Tabela (DataFrame) 2:
Dimensões: (3, 2) (Linhas x Colunas)

--- PRÉVIA DO CABEÇALHO ---
       Estatísticas         Estatísticas.1
0  Total de gol(o)s         